# 임차 농지

In [31]:
import pandas as pd
import json

df = pd.read_csv("../data/raw/임차농지_현황.csv", encoding="cp949")
print(df.head())

   시도코드     시도  시군구  필지수  면적(ha)
0    11  서울특별시  강남구   50     7.8
1    11  서울특별시  강동구   50     6.4
2    11  서울특별시  강서구  136    30.9
3    11  서울특별시  구로구   10     1.5
4    11  서울특별시  노원구    1     0.4


In [32]:
sido_mapping = {
    '전라남도': '전남',
    '전북특별자치도': '전북',
    '전라북도': '전북',
    '강원특별자치도': '강원',
    '강원도': '강원',
    '경기도': '경기',
    '경상남도': '경남',
    '경상북도': '경북',
    '충청남도': '충남',
    '충청북도': '충북'
}

# 서울/광역시 목록 (구 통합 대상)
metropolitan_cities = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', 
                      '광주광역시', '대전광역시', '울산광역시']

def convert_region_name(sido, sigungu):
    """지역명을 지침에 따라 변환하는 함수"""
    
    # 서울/광역시 - 구 통합 (풀네임 유지)
    if sido in metropolitan_cities:
        return sido
    
    # 세종특별자치시 - 간단히 표기
    elif sido == '세종특별자치시':
        return '세종시'
    
    # 제주특별자치도 - 제주시/서귀포시로 분리
    elif sido == '제주특별자치도':
        if '제주시' in sigungu:
            return '제주시'
        elif '서귀포시' in sigungu:
            return '서귀포시'
        else:
            return '제주시'  # 기본값
    
    # 도 단위 - 간단히 표기하고 시군구 포함
    else:
        short_sido = sido_mapping.get(sido, sido)
        
        # 시 단위 구 통합 처리 (예: "전주시 덕진구" -> "전주시")
        if '구' in sigungu and '시' in sigungu:
            city_name = sigungu.split()[0] if ' ' in sigungu else sigungu.replace('구', '').replace('시', '') + '시'
            return f"{short_sido} {city_name}"
        else:
            return f"{short_sido} {sigungu}"

df['지역'] = df.apply(lambda row: convert_region_name(row['시도'], row['시군구']), axis=1)

In [33]:
# 지역별 그룹화 및 집계
result_df = df.groupby('지역').agg({
    '필지수': 'sum',
    '면적(ha)': 'sum'
}).reset_index()

# 컬럼명 변경
result_df.columns = ['지역', '대지의 개수 [필지수]', '면적 [ha]']

# 면적 소수점 1자리로 반올림
result_df['면적 [ha]'] = result_df['면적 [ha]'].round(1)

# 지역명 순으로 정렬
result_df = result_df.sort_values('지역').reset_index(drop=True)

In [34]:
with open('../docs/data/임차농지_현황.json', 'w', encoding='utf-8') as f:
    json.dump(result_df.to_dict('records'), f, ensure_ascii=False, indent=2)


result_df.to_csv('../data/임차농지_현황.csv', index=False, encoding='utf-8-sig')

print(f"✓ 파일 저장 완료")
# 저장된 파일 확인
print(result_df.head())

✓ 파일 저장 완료
       지역  대지의 개수 [필지수]  면적 [ha]
0  강원 강릉시         11959   2213.7
1  강원 고성군          8327   1774.2
2  강원 동해시          1728    201.6
3  강원 삼척시          6432   1241.6
4  강원 속초시           725    111.1
